# Visualising Eye Movement

In [24]:
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import numpy as np

import matplotlib
matplotlib.use("Agg") 

In [25]:
import sys
import os

sys.path.append(os.path.abspath("../src"))  # Add folder_a to sys.path
from config import *

In [26]:
experiment = "KING_DEVICK"
participant_ids = pd.read_parquet(
        f"{PREPROCESSED_DIR}/{experiment}_samples.pq", 
        columns=["participant_id"]
    )

participant_ids = participant_ids["participant_id"].unique()

In [27]:
# Read data for participant
participant_id = participant_ids[11]
filters = [('participant_id', '=', str(participant_id))]

samples_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_samples.pq",
    filters=[('participant_id', '=', participant_id)]
    )

events_df = pd.read_parquet(
    f"{PREPROCESSED_DIR}/{experiment}_events.pq",
    filters=[('participant_id', '=', participant_id)]
    )

In [28]:
events_df

,index,experiment,participant_id,trial_id,time,event,eye,start_time,end_time,duration,...,y,avg_pupil_size,start_x,start_y,end_x,end_y,amplitude,peak_velocity,marks,time_elapsed
0,174330,KING_DEVICK,143,0,0,START,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,12.491893
1,174562,KING_DEVICK,143,0,30,EFIX,L,30.0,188.0,158.0,...,231.0,3473.0,NaN,NaN,NaN,NaN,NaN,NaN,0,12.491893
2,174563,KING_DEVICK,143,0,30,EFIX,R,30.0,189.0,159.0,...,179.7,3650.0,NaN,NaN,NaN,NaN,NaN,NaN,0,12.491893
3,174565,KING_DEVICK,143,0,189,ESACC,L,189.0,257.0,68.0,...,NaN,NaN,505.8,218.3,251.0,183.1,4.61,331.0,0,12.491893
4,174564,KING_DEVICK,143,0,190,ESACC,R,190.0,246.0,56.0,...,NaN,NaN,524.6,170.6,295.4,130.2,4.18,319.0,0,12.491893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,176335,KING_DEVICK,143,2,32851,EFIX,L,32851.0,32901.0,50.0,...,163.8,2064.0,NaN,NaN,NaN,NaN,NaN,NaN,0,31.086179
968,176339,KING_DEVICK,143,2,32902,ESACC,L,32902.0,33106.0,204.0,...,NaN,NaN,113.7,247.6,540.5,156.4,7.94,0.0,0,31.086179
969,176340,KING_DEVICK,143,2,32909,ESACC,R,32909.0,33106.0,197.0,...,NaN,NaN,523.4,198.2,531.6,174.3,0.46,104.0,0,31.086179
970,175900,KING_DEVICK,143,2,33004,TRIAL_VAR_DATA,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,31.086179


In [29]:
trial_id=1

sample_df = samples_df.loc[samples_df["trial_id"]==trial_id,:]

# Extract fixpoints
# event_df = events_df.loc[events_df["trial_id"]==trial_id,:]
# fixpoints_df = event_df[event_df["event"]=="FIXPOINT"].loc[:,["participant_id", "trial_id", "time", "event", "colour", "stimulus_x", "stimulus_y"]]

# Insert fixpoints in sample data
# Ensure you are modifying actual copies
sample_df = sample_df.copy()
# fixpoints_df = fixpoints_df.copy()

# Sort by participant_id, trial_id, then time (important for merge_asof with 'by')
sample_df = sample_df.sort_values(["time"])
# fixpoints_df = fixpoints_df.sort_values(["time"])

# Rename 'colour' column to 'fixpoint' so it's ready to merge
# fixpoints_df = fixpoints_df.rename(columns={"colour": "fixpoint"})

# Perform a backward-looking join: for each row in sample_df, find the most recent fixpoint time
# sample_df = pd.merge_asof(
#     sample_df,
#     fixpoints_df,
#     on=["time"],
#     by=["participant_id", "trial_id"],
#     direction="nearest",
#     tolerance=10
# )

# sample_df["fixpoint"] = sample_df["fixpoint"].map({RED:"red", GREEN:"green", BLUE:"blue", WHITE:"white"})

In [30]:
x = sample_df.copy()
x_sampled = x[x.index % 100 == 0]

# Apply Seaborn style
sns.set(style="whitegrid")

fig, ax = plt.subplots(1, 1, figsize = (6, 6))

# Initialize the scatter plot object
scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
# scatter_fixpoint = ax.scatter([], [], label="Fixpoint", alpha=0.6, s=50)  # 's' is for size, 'c' is for color

x_min, x_max = 0, 1920 
y_min, y_max = 0, 1080 
ax.set_xlim([x_min, x_max]) 
ax.set_ylim([y_min, y_max]) 

# Add legend to distinguish between the three points
ax.legend()

def animate(i):
    # Update the scatter plot data for each frame
    
    # Update coordinates
    coords_right = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_right"].iloc[i]]
    coords_left = np.c_[x_sampled["x_left"].iloc[i], x_sampled["y_left"].iloc[i]] 
    # coords_fixpoint = np.c_[x_sampled["stimulus_x"].iloc[i], x_sampled["stimulus_y"].iloc[i]] 

    scatter_right.set_offsets(coords_right)    
    scatter_left.set_offsets(coords_left)
    # scatter_fixpoint.set_offsets(coords_fixpoint)

    # Update colour based on fixpoint
    # fixpoint_color = x_sampled["fixpoint"].iloc[i]
    # if pd.isna(fixpoint_color):
    #     fixpoint_color = "gray"
    # elif fixpoint_color == "white":
    #     fixpoint_color = "black"
    # scatter_fixpoint.set_color([fixpoint_color])

    return scatter_right, scatter_left#, scatter_fixpoint

anim = animation.FuncAnimation(
    fig, 
    animate, 
    frames = len(x_sampled), 
    interval=100,
    blit = False
)

anim.save(f"Animations/{experiment}_{participant_id}_{trial_id}.mp4", writer="ffmpeg", fps=30)



/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_6808/3939875487.py:10: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_right = ax.scatter([], [], cmap='viridis', label="Right", alpha=0.6)  # 's' is for size, 'c' is for color
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_6808/3939875487.py:11: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  scatter_left = ax.scatter([], [], cmap='coolwarm', label="Left", alpha=0.6)  # 's' is for size, 'c' is for color
2025-04-25 13:09:34,487 - INFO - animation.save:1076 - Animation.save using <class 'matplotlib.animation.FFMpegWriter'>
2025-04-25 13:09:34,488 - INFO - animation._run:319 - MovieWriter._run: running command: ffmpeg -f rawvideo -vcodec rawvideo -s 600x600 -pix_fmt rgba -framerate 30 -loglevel error -i pipe: -vcodec h264 -pix_fmt yuv420p -y Animations/KING_DEVICK_143_1.mp4
